In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from diffusers import FluxInpaintPipeline
from diffusers.utils import load_image
from pipeline import create_low_vram_flux_pipeline

In [ ]:
device = "cuda:0"
pipeline = create_low_vram_flux_pipeline(device, factory=FluxInpaintPipeline)

In [ ]:
prompt = "a vibrant and colorful scene. There is a yellow wall that has a rustic charm, with visible signs of age and weathering. The wall is adorned with a wooden door that has a rich, dark brown color, adding a touch of rustic elegance to the scene. The door is closed, and there is a small window located above it, which is also made of wood and has a similar dark brown color. The wall and door are set against a clear blue sky, which is dotted with a few clouds, suggesting a bright and sunny day. The overall composition of the background creates a warm and inviting atmosphere. "
# prompt = "Yellow wall with a wooden door and a wooden window"
img_url = "./assets/car.png"
mask_url = "./assets/car_mask.png"
source = load_image(img_url)
mask = load_image(mask_url)
width = source.size[0]
height = source.size[1]
display(source)
display(mask)

In [ ]:
import cv2
import numpy as np
from PIL import Image
# Dilate the mask to make the inpainting more robust
mask = cv2.dilate(np.array(mask), np.ones((7, 7), np.uint8), iterations=5)
mask = Image.fromarray(mask)
display(mask)

In [ ]:
# Show masked image
masked = np.array(source.copy())
mask_cv2 = np.array(mask)
masked = masked * (mask_cv2 == 0)
masked = Image.fromarray(masked)
display(masked)

In [ ]:
res = pipeline(prompt=prompt, image=source, mask_image=mask, width=width, height=height, strength=1.0)

In [ ]:
res.images[0]

In [ ]:
def show_latent(self, latents):
    latents = self._unpack_latents(latents, height, width, self.vae_scale_factor)
    latents = (latents / self.vae.config.scaling_factor) + self.vae.config.shift_factor
    image = self.vae.decode(latents, return_dict=False)[0]
    image = self.image_processor.postprocess(image, output_type="pil")[0]
    image.save("latent.png")
    display(image)
    
pipeline.show_latent = show_latent.__get__(pipeline)